In [1]:
%cd '/home/vri/Projects/research/Flood-detection'

# necessary for cross directory imports. 
# replace path with root dir of projects

/home/vri/Projects/research/Flood-detection


In [2]:
import os
import json
import glob
import pandas as pd
import matplotlib.pyplot as plt
import tarfile
import numpy as np 

import subprocess
from subprocess import PIPE

import rasterio
import cv2
from rasterio.plot import show

import torch
import torchvision
from torch import nn , utils, Tensor
from torchvision import transforms as T
import torch.utils.data as data
import torchvision.utils as vutils
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import models
import pytorch_lightning as pl
from pytorch_lightning import callbacks
from pytorch_lightning.loggers import TensorBoardLogger
import torch.optim as optim

#module for preprocessing data directory structure
from utils import utils_dirs

In [3]:
torch.backends.cudnn.benchmark = True
torch.set_float32_matmul_precision('medium')

In [4]:
ROOT_DIR = '/home/vri/Projects/research/Flood-detection/data/transformer_data/sen12floods_s2_source'
LABEL_PATH = './data/transformer_labels/sen12floods_s2_labels'

In [5]:
#list of valid data paths
actual_flist = utils_dirs.get_img_folders(flag='valid', root_path = ROOT_DIR)

The number of valid image subfolders are: 1949


In [6]:
def get_image_labels(dirname, label_dir):
    """function to get image label for corresponding image dir

    Args:
        dirname (str): name of image subdirectory
        label_dir (path.path): path to label directory
    """
    pd = dirname.split("_")
    pd = f"{pd[0]}_{pd[1]}_labels_{pd[3]}_{pd[4]}_{pd[5]}_{pd[6]}"

    json_data = open (f"{label_dir}/{pd}/stac.json", "rb")
    jdata = json.load(json_data)
    flood = jdata["properties"]["FLOODING"]

    image_label = 1 if flood else 0

    return image_label


In [7]:
class CustomDataset(Dataset):
    """pytorch custom dataset class. 
    Inherits from torch.utils.data.Dataset

    Args:
        img_list (list): list of valid image paths (flist)
        label_path (pathlib.path) : path to labels dir
        transform (torchvision.transforms): image transforms
    """

    def __init__(self, img_list, label_path, transform = None ):
        self.transform = transform
        self.img_list = img_list
        self.label_path = label_path
        

    def __getitem__(self, idx):
        img_name = utils_dirs.get_dirname(self.img_list[idx])
        label = get_image_labels(img_name, self.label_path)

        img = cv2.imread(f"{self.img_list[idx]}/stack.tif")
        if self.transform:
            img = self.transform(img)
        
        return img,label
    
    def __len__(self):
        return len(self.img_list)

In [8]:
BATCH_SIZE = 32
MEAN, STD=(0.485, 0.456, 0.406), (0.229, 0.224, 0.225)

In [9]:
data_transforms = T.Compose([
    T.ToTensor() ,
    T.RandomHorizontalFlip(p = 0.5) ,
    T.RandomVerticalFlip (p = 0.5) ,
    T.Resize((224, 224), antialias=True),
    T.Normalize(MEAN, STD)
])


In [10]:
total_dataset = CustomDataset(actual_flist, LABEL_PATH, data_transforms)

test_size = int(0.15 * len(total_dataset))

train_size = int(0.8*(len(total_dataset) - test_size))
val_size = len(total_dataset)- test_size - train_size

train_set , test_set, val_set = random_split(total_dataset, [train_size, test_size, val_size])

print(f"Images in train set: {len(train_set)}")
print(f"Images in test set: {len(test_set)}")
print(f"Images in val set: {len(val_set)}")

Images in train set: 1325
Images in test set: 292
Images in val set: 332


In [11]:
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=16)
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, num_workers=16)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, num_workers=16)

In [12]:
next(iter(train_loader))[0].shape

torch.Size([32, 3, 224, 224])

------------------

- Resnet18 Model

In [13]:
class CustomResNet18(nn.Module):
    """Custom Resnet-18 class.
    Inherits from torch.nn.Module
    Args:
        num_classes (int): number of classes
    """
    def __init__(self, num_classes = 2):
        super(CustomResNet18, self).__init__()
        self.model = models.resnet18(weights = models.ResNet18_Weights.DEFAULT)

        for param in self.model.parameters():
            param.requires_grad = False
        
        num_features = self.model.fc.in_features
        self.model.fc = nn.Sequential(
            nn.Linear(num_features, num_classes),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.model(x)

In [14]:
class ResNetBinaryClassifier(pl.LightningModule):
    """ Pytorch lightning module for Resnet Binary Classifier
    """

    def __init__(self, num_classes:int, config = None):
        super(ResNetBinaryClassifier, self).__init__()
        self.resnet_model = CustomResNet18(num_classes= 2)
        self.config = config
        
    def forward(self, img_batch):
        return self.resnet_model(img_batch)
    
    def common_step(self, batch):
        x, y = batch
        logits = self.forward (x)

        loss_fn = nn.CrossEntropyLoss()
        loss = loss_fn(logits, y)

        correct = logits.argmax(dim=1).eq(y).sum().item()
        accuracy = self.accuracy(correct, y)

        return loss, accuracy

    def accuracy(self, correct, labels):
        return correct / len(labels)

    def training_step(self, batch, batch_idx):
        loss, accuracy = self.common_step(batch)
        
        self.log_dict({
            "train_loss": loss,
            "train_accuracy" : accuracy,
        }, prog_bar=True)

        return loss

    def validation_step(self, batch, batch_idx):
        loss, accuracy = self.common_step(batch)

        self.log('val_loss', loss, prog_bar=True)
        self.log('val_accuracy', accuracy, prog_bar=True)

        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self.forward(x)
        correct = logits.argmax(dim=1).eq(y).sum().items()
        total = len(y)
        accuracy = correct / total

        self.log('test_accuracy', accuracy, prog_bar=True)
        return accuracy


    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr = self.config['lr'])
        return optimizer


In [15]:
config = {
    'lr' : 1e-3
}

In [16]:
def train_resnet(config, num_epochs = 10, checkpoint = None):
    """function to fit pytorch lightning trainer

    Args:
        config (dict): configuration values
        num_epochs (int): Number of epochs. Defaults to 10.
        checkpoint (_type_): model checkpoint if training from a saved point. Defaults to None.

    Returns:
        trainer: pl.trainer
    """
    model = ResNetBinaryClassifier(2, config)
    tlogger = TensorBoardLogger(save_dir="flood-detection-logs", name="resnet-model", version = "v1")

    resnet_callbacks = [
        # early_stopping,
        callbacks.ModelCheckpoint(monitor='val_loss', save_top_k=1,
                                  save_on_train_epoch_end= False,
                                  filename='{epoch}-{val_loss:.2f}')
    ]


    trainer = pl.Trainer(accelerator= "gpu",
                         logger= tlogger,
                         log_every_n_steps= 2,
                         precision=16,
                         enable_checkpointing= True,
                         callbacks= resnet_callbacks ,
                         devices= 1,
                         enable_progress_bar= True,
                         max_epochs= num_epochs
                         )
    
    trainer.fit(model, train_dataloaders=train_loader, val_dataloaders= val_loader, ckpt_path = checkpoint)
    return trainer
    

In [17]:
resnet_trainer = train_resnet(config= config, num_epochs= 10)

/home/vri/miniconda3/envs/raster_env/lib/python3.11/site-packages/lightning_fabric/connector.py:562: UserWarning: 16 is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
  rank_zero_warn(
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type           | Params
------------------------------------------------
0 | resnet_model | CustomResNet18 | 11.2 M
------------------------------------------------
1.0 K     Trainable params
11.2 M    Non-trainable params
11.2 M    Total params
44.710    Total estimated model params size (MB)


Epoch 9: 100%|██████████| 42/42 [00:08<00:00,  4.70it/s, v_num=v1, train_loss=0.313, train_accuracy=1.000, val_loss=0.313, val_accuracy=1.000]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 42/42 [00:08<00:00,  4.70it/s, v_num=v1, train_loss=0.313, train_accuracy=1.000, val_loss=0.313, val_accuracy=1.000]


In [18]:
resnet_trainer.validate(dataloaders=test_loader, ckpt_path='best')

Restoring states from the checkpoint path at flood-detection-logs/resnet-model/v1/checkpoints/epoch=9-val_loss=0.31.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at flood-detection-logs/resnet-model/v1/checkpoints/epoch=9-val_loss=0.31.ckpt


Validation DataLoader 0: 100%|██████████| 10/10 [00:00<00:00, 33.80it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Runningstage.validating metric      DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      val_accuracy                  1.0
        val_loss            0.3133895993232727
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'val_loss': 0.3133895993232727, 'val_accuracy': 1.0}]

-----------

- Vision Transformer

In [19]:
class ViTModule(nn.Module):
    def __init__(self, num_classes=2):
        super(ViTModule, self).__init__()
        self.model=models.vit_b_16(weights=models.ViT_B_16_Weights.DEFAULT)

    def forward(self, x):
        return self.model(x)

In [20]:
class ViTClassifier(pl.LightningModule):
    def __init__(self,config = None):
        super(ViTClassifier, self).__init__()
        self.resnet_model = ViTModule()
        self.config = config
        
    def forward(self, img_batch):
        return self.resnet_model(img_batch)
    
    def common_step(self, batch):
        x, y = batch
        logits = self.forward (x)

        loss_fn = nn.CrossEntropyLoss()
        loss = loss_fn(logits, y)

        correct = logits.argmax(dim=1).eq(y).sum().item()
        accuracy = self.accuracy(correct, y)

        return loss, accuracy

    def accuracy(self, correct, labels):
        return correct / len(labels)

    def training_step(self, batch, batch_idx):
        loss, accuracy = self.common_step(batch)
        
        self.log_dict({
            "train_loss": loss,
            "train_accuracy" : accuracy,
        }, prog_bar=True)

        return loss

    def validation_step(self, batch, batch_idx):
        loss, accuracy = self.common_step(batch)

        self.log('val_loss', loss, prog_bar=True)
        self.log('val_accuracy', accuracy, prog_bar=True)

        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self.forward(x)
        correct = logits.argmax(dim=1).eq(y).sum().items()
        total = len(y)
        accuracy = correct / total

        self.log('test_accuracy', accuracy, prog_bar=True)
        return accuracy


    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr = self.config['lr'])
        return optimizer


In [21]:
vit_config ={
    'lr': 1e-3
}

In [22]:
def train_vit(config, num_epochs = 10, checkpoint = None):
    model = ViTClassifier( config)
    tlogger = TensorBoardLogger(save_dir="flood-detection-logs", name="vit-model", version = "v1")

    vit_callbacks = [
        # early_stopping,
        callbacks.ModelCheckpoint(monitor='val_loss', save_top_k=1,
                                  save_on_train_epoch_end= False,
                                  filename='{epoch}-{val_loss:.2f}')
    ]


    trainer = pl.Trainer(accelerator= "gpu",
                         logger= tlogger,
                         log_every_n_steps= 2,
                         precision=16,
                         enable_checkpointing= True,
                         callbacks= vit_callbacks ,
                         devices= 1,
                         enable_progress_bar= True,
                         max_epochs= num_epochs
                         )
    
    trainer.fit(model, train_dataloaders=train_loader, val_dataloaders= val_loader, ckpt_path = checkpoint)
    return trainer
    

In [23]:
vit_trainer = train_vit(config= vit_config, num_epochs= 10)

/home/vri/miniconda3/envs/raster_env/lib/python3.11/site-packages/lightning_fabric/connector.py:562: UserWarning: 16 is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
  rank_zero_warn(
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type      | Params
-------------------------------------------
0 | resnet_model | ViTModule | 86.6 M
-------------------------------------------
86.6 M    Trainable params
0         Non-trainable params
86.6 M    Total params
346.271   Total estimated model params size (MB)


Epoch 9: 100%|██████████| 42/42 [00:18<00:00,  2.28it/s, v_num=v1, train_loss=7.46e-6, train_accuracy=1.000, val_loss=7.44e-6, val_accuracy=1.000]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 42/42 [00:18<00:00,  2.28it/s, v_num=v1, train_loss=7.46e-6, train_accuracy=1.000, val_loss=7.44e-6, val_accuracy=1.000]


In [24]:
vit_trainer.validate(dataloaders=test_loader, ckpt_path='best')

Restoring states from the checkpoint path at flood-detection-logs/vit-model/v1/checkpoints/epoch=9-val_loss=0.00.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at flood-detection-logs/vit-model/v1/checkpoints/epoch=9-val_loss=0.00.ckpt


Validation DataLoader 0: 100%|██████████| 10/10 [00:00<00:00, 11.90it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Runningstage.validating metric      DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      val_accuracy                  1.0
        val_loss           7.442823971359758e-06
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'val_loss': 7.442823971359758e-06, 'val_accuracy': 1.0}]

----------------

- SWIN Transformer

In [25]:
class SwinModule(nn.Module):
    """class for tiny Swin transformer v2"""
    def __init__(self, num_classes=2):
        super(SwinModule, self).__init__()
        self.model=models.swin_v2_t(weights=models.Swin_V2_T_Weights.DEFAULT)

    def forward(self, x):
        return self.model(x)

In [26]:
class SWINClassifier(pl.LightningModule):
    def __init__(self,config = None):
        super(SWINClassifier, self).__init__()
        self.swin_model = SwinModule()
        self.config = config
        
    def forward(self, img_batch):
        return self.swin_model(img_batch)
    
    def common_step(self, batch):
        x, y = batch
        logits = self.forward (x)

        loss_fn = nn.CrossEntropyLoss()
        loss = loss_fn(logits, y)

        correct = logits.argmax(dim=1).eq(y).sum().item()
        accuracy = self.accuracy(correct, y)

        return loss, accuracy

    def accuracy(self, correct, labels):
        return correct / len(labels)

    def training_step(self, batch, batch_idx):
        loss, accuracy = self.common_step(batch)
        
        self.log_dict({
            "train_loss": loss,
            "train_accuracy" : accuracy,
        }, prog_bar=True)

        return loss

    def validation_step(self, batch, batch_idx):
        loss, accuracy = self.common_step(batch)

        self.log('val_loss', loss, prog_bar=True)
        self.log('val_accuracy', accuracy, prog_bar=True)

        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self.forward(x)
        correct = logits.argmax(dim=1).eq(y).sum().items()
        total = len(y)
        accuracy = correct / total

        self.log('test_accuracy', accuracy, prog_bar=True)
        return accuracy


    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr = self.config['lr'])
        return optimizer


In [27]:
swin_config ={
    'lr': 1e-3
}

In [28]:
def train_swin(config, num_epochs = 10, checkpoint = None):
    model = SWINClassifier( config)
    tlogger = TensorBoardLogger(save_dir="flood-detection-logs", name="swin-model", version = "v1")

    vit_callbacks = [
        # early_stopping,
        callbacks.ModelCheckpoint(monitor='val_loss', save_top_k=1,
                                  save_on_train_epoch_end= False,
                                  filename='{epoch}-{val_loss:.2f}')
    ]


    trainer = pl.Trainer(accelerator= "gpu",
                         logger= tlogger,
                         log_every_n_steps= 2,
                         precision=16,
                         enable_checkpointing= True,
                         callbacks= vit_callbacks ,
                         devices= 1,
                         enable_progress_bar= True,
                         max_epochs= num_epochs
                         )
    
    trainer.fit(model, train_dataloaders=train_loader, val_dataloaders= val_loader, ckpt_path = checkpoint)
    return trainer
    

In [29]:
swin_trainer = train_swin(config= swin_config, num_epochs= 10)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type       | Params
------------------------------------------
0 | swin_model | SwinModule | 28.4 M
------------------------------------------
28.4 M    Trainable params
0         Non-trainable params
28.4 M    Total params
113.406   Total estimated model params size (MB)


Epoch 9: 100%|██████████| 42/42 [00:16<00:00,  2.47it/s, v_num=v1, train_loss=7.98e-7, train_accuracy=1.000, val_loss=0.000, val_accuracy=1.000]    

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 42/42 [00:16<00:00,  2.47it/s, v_num=v1, train_loss=7.98e-7, train_accuracy=1.000, val_loss=0.000, val_accuracy=1.000]


In [30]:
swin_trainer.validate(dataloaders=test_loader, ckpt_path='best')

Restoring states from the checkpoint path at flood-detection-logs/swin-model/v1/checkpoints/epoch=3-val_loss=0.00.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at flood-detection-logs/swin-model/v1/checkpoints/epoch=3-val_loss=0.00.ckpt


Validation DataLoader 0: 100%|██████████| 10/10 [00:00<00:00, 12.58it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Runningstage.validating metric      DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      val_accuracy                  1.0
        val_loss                    0.0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'val_loss': 0.0, 'val_accuracy': 1.0}]

------------

- Trial Logs

In [31]:
#visualise metric plots via tensorboard
#ensure tensorflow is installed on local device and path to logdir is accurate

%reload_ext tensorboard
%tensorboard --logdir="/home/vri/Projects/research/Flood-detection/flood-detection-logs" --host localhost --port 8081